In [48]:
# Boundary condition taken care of are very simple and are as follows:
#  1) inlet u
#  2) outlet u
#  3) inlet p
#  4) outlet p
#  5) stagnation pressure BEFORE the inlet
# Present code can take care of 'Stagnation pressure', only before 'inlet'  
# Mass flow rate should be given to guess initial velocity field

In [116]:
import numpy as np
from scipy import linalg

In [117]:
# setting up the grid geometery
geometery_name = 'nozzle'
area_profile = 'linear'
length = 10.0  # in meter

u_grid_pts = 4
p_grid_pts = u_grid_pts + 1
total_grid_pts = u_grid_pts + p_grid_pts

no_of_cv = (total_grid_pts - 3) / 2
delta_x = length / no_of_cv

grid = np.zeros(total_grid_pts)

if area_profile == 'linear':
    A_inlet = 0.5
    A_outlet = 0.1
    # for converging nozzle
    A = np.linspace(np.fmin(A_outlet, A_inlet), np.fmax(A_inlet, A_outlet), total_grid_pts)[::-1] 
else:
    A = np.zeros(total_grid_pts)
    A += 1.0


print(grid, '\n')
print(A)

[ 0.  0.  0.  0.  0.  0.  0.  0.  0.] 

[ 0.5   0.45  0.4   0.35  0.3   0.25  0.2   0.15  0.1 ]


In [118]:
# setting up the fluid properties
row = 1.0 # kg/m3
mu = 0.0 # N/m-s
# acc. due to gravity
g = 0.0 # m/sec2

In [119]:
# given initial values
mass_flow_rate = ('given', 1.) # kg/sec.
alpha = 0.4 # under relaxation parameter
n = 1000 # no. of iterations

In [120]:
# given B.Cs.
inlet_u = ('not_given', 0.0)
outlet_u = ('not_given', 5.0)
inlet_p = ('not_given', 0.0)
outlet_p = ('given', 0.0)
stag_p = ('given', 10.0) # stagnation pressure "before" the 'inlet'

In [121]:
# declaring u velocity coefficients
F = np.zeros(p_grid_pts)
D = np.zeros(p_grid_pts)
aE = np.zeros(u_grid_pts)
aW = np.zeros(u_grid_pts)
aP = np.zeros(u_grid_pts)
delta_p = np.zeros(u_grid_pts)
u_source = np.zeros(u_grid_pts)

In [122]:
# extracting values of u - velocity from main grid
u_values = grid[1::2]
# initializing the u-velocity grid
if mass_flow_rate[0] == 'given':
    u_values[:] = mass_flow_rate[1] / (row * A[1::2]) 
elif inlet_u[0] == 'given' and outlet_u[0] == 'given':
    u = np.linspace(np.fmax(inlet_u[1], outlet_u[1]), np.fmin(inlet_u[1], outlet_u[1]), u_grid_pts)
    u_values[:] = u
elif inlet_u[0] == 'given':
    u = np.linspace(inlet_u[1], inlet_u[1] - 2, u_grid_pts)
    u_values[:] = u
elif outlet_u[0] == 'given':
    u = np.linspace(outlet_u[1], outlet_u[1] - 2, u_grid_pts)[::-1]
    u_values[:] = u

# extracting pressure values from main grid points
p_values = grid[0::2]
# initialising pressure values
if stag_p[0] == 'given' and outlet_p[0] == 'given':
    p_inlet = stag_p[1] - 0.5 * row * (u_values[0] ** 2) * (A[1] / A[0]) ** 2
    p = np.linspace(outlet_p[1], stag_p[1], p_grid_pts)[::-1]
    p_values[:] = p
    #p_values[0] = p_inlet
elif inlet_p[0] == 'given' and outlet_p[0] == 'given':
    p = np.linspace(np.fmax(inlet_p[1], outlet_p[1]), np.fmin(inlet_p[1], outlet_p[1]), p_grid_pts)
    p_values[:] = p
elif inlet_p[0] == 'given':
    p = np.linspace(0.0, inlet_p[1], p_grid_pts)[::-1]
    p_values[:] = p
elif outlet_p[0] == 'given':
    p = np.linspace(0.0, outlet_p[1], p_grid_pts)
    p_values[:] = p

print(u_values,'\n')
print(p_values,'\n')

[ 2.22222222  2.85714286  4.          6.66666667] 

[ 10.    7.5   5.    2.5   0. ] 



In [123]:
for i in range(n):
    
    # mass flow rate at boundary scalar point
    F[0] = row * (u_values[0] * A[1] / A[0]) * A[0]
    
    # mass flow rate at inner scalar points
    F[1: p_grid_pts - 1] = A[2: total_grid_pts - 2: 2] * row * (u_values[0: u_grid_pts - 1] + u_values[1:]) / 2.0
    
    # mass flow rate for outermost scalar point .........
    if i == 0 and mass_flow_rate[0] == 'given':
        F[p_grid_pts - 1] = mass_flow_rate[1]
    else:
        F[p_grid_pts - 1] = A[total_grid_pts - 2] * row * u_values[u_grid_pts - 1]
    # .............................................
    
    D[:] = A[:: 2] * mu / delta_x

    #print(F, '\n')
    #print(D, '\n')

    delta_p = (p_values[0:p_grid_pts - 1] - p_values[1:]) * A[1::2]

    aE[:u_grid_pts - 1] = D[1:p_grid_pts - 1] + np.fmax(-F[1:p_grid_pts - 1], 0.0)

    aW[1:] = D[1:p_grid_pts - 1] + np.fmax(F[1:p_grid_pts - 1], 0.0)

    if stag_p[0] == 'given':
        aP[1:] = np.fmax(F[2:], 0.0) + np.fmax(-F[1:p_grid_pts - 1], 0.0) + D[2:] + D[1:p_grid_pts - 1]
        aP[0] = F[1] + F[0] * 0.5 * (A[1] / A[0]) ** 2
    else:
        aP[:] = np.fmax(F[1:], 0.0) + np.fmax(-F[0:p_grid_pts - 1], 0.0) + D[1:] + D[0:p_grid_pts - 1]


    de = A[1::2] / aP # for pressure correction

    #print(aE, '\n')
    #print(aW, '\n')
    #print(aP, '\n')
    #print(aP1, '\n')
    #print(u_source, '\n')

    # constructing u TDMA
    u_tdma = np.diag(aP[:], k=0) + np.diag(-aE[:-1], k=1) + np.diag(-aW[1:], k=-1)
    
    # constructing u source
    if stag_p[0] == 'given':
        u_source[0] = (stag_p[1] - p_values[1]) * A[1] + F[0] * A[1] / A[0] * u_values[0]
        u_source[1:] = delta_p[1:] + row * g
    else:
        u_source[:] = delta_p[:] + row * g
    
    # implementing u boundary conditions on u-TDMA and u-source term
    if inlet_u[0] =='given' and outlet_u[0] == 'given':
        u_tdma[0] = 0.0
        u_tdma[0, 0] = 1.0
        u_source[0] = inlet_u[1]
        
        u_tdma[u_grid_pts - 1] = 0.0
        u_tdma[u_grid_pts - 1, u_grid_pts - 1] = 1.0
        u_source[u_grid_pts - 1] = outlet_u[1]
        
    elif inlet_u[0] =='given':
        u_tdma[0] = 0.0
        u_tdma[0, 0] = 1.0
        u_source[0] = inlet_u[1]
        
    elif outlet_u[0] =='given':
        u_tdma[u_grid_pts - 1] = 0.0
        u_tdma[u_grid_pts - 1, u_grid_pts - 1] = 1.0
        u_source[u_grid_pts - 1] = outlet_u[1]

    #print(u_tdma)
    #print(u_source)

    # solution after solving u TDMA
    u_solution = linalg.solve(u_tdma, u_source)

    #print(u_solution)
    
    # solving for pressure correction...................................................
    p_source = np.zeros(p_grid_pts)
    aE_p = np.zeros(p_grid_pts)
    aW_p = np.zeros(p_grid_pts)

    aE_p[0:p_grid_pts - 1] = row * de[:] * A[1::2]
    aW_p[1:] = row * de[:] * A[1::2]
    aP_p = aE_p + aW_p

    #print(aE_p, '\n')
    #print(aW_p, '\n')
    #print(aP_p, '\n')
    
    # constructing p_tdma matrix
    p_tdma = np.diag(aP_p[:], k=0) + np.diag(-aE_p[:-1], k=1) + np.diag(-aW_p[1:], k=-1)
    
    # constructing p_source column vector
    p_source[1:p_grid_pts - 1] = row * ((u_solution[:u_grid_pts - 1] * A[1:total_grid_pts - 3:2]) - (u_solution[1:] * A[3:total_grid_pts - 1:2])) 

    # implementing B.C. on pressure correction tdma
    # nodes at which pressure is known, one need not to solve for pressure correction at those nodes,
    # thus one has to set pressure correction to zero at those nodes.
    
    if stag_p[0] == 'given' and outlet_p[0] == 'given':
        p_tdma[0] = 0.0
        p_tdma[p_grid_pts - 1] = 0.0
        p_tdma[p_grid_pts - 1, p_grid_pts - 1] = 1.0
        p_tdma[0, 0] = 1.0
        
        p_source[0] = 0.0
        p_source[p_grid_pts - 1] = 0.0
    
    elif inlet_p[0] == 'given' and outlet_p[0] == 'given':
        p_tdma[0] = 0.0
        p_tdma[p_grid_pts - 1] = 0.0
        p_tdma[p_grid_pts - 1, p_grid_pts - 1] = 1.0
        p_tdma[0, 0] = 1.0
        
        p_source[0] = 0.0
        p_source[p_grid_pts - 1] = 0.0
    
    elif inlet_p[0] == 'given':
        p_tdma[0] = 0.0
        p_tdma[0, 0] = 1.0
        
        p_source[0] = 0.0
        
    elif outlet_p[0] == 'given':
        p_tdma[p_grid_pts - 1] = 0.0
        p_tdma[p_grid_pts - 1, p_grid_pts - 1] = 1.0
        
        p_source[p_grid_pts - 1] = 0.0
        
    # constructing source term with outlet B.C. of Pressure
    #p_source[1:p_grid_pts - 1] = row * ((u_solution[:u_grid_pts - 1] * A[1:total_grid_pts - 3:2]) - (u_solution[1:] * A[3:total_grid_pts - 1:2])) 

    #print(p_tdma, '\n')
    #print(p_source, '\n')

    # solution for pressure
    p_solution = linalg.solve(p_tdma, p_source)
    #print(p_solution)

    # u-velocity correction
    u_solution[:] += de[:] * (p_solution[:p_grid_pts - 1] - p_solution[1:])
    #print(u_solution)

    # correcting pressure
    if stag_p[0] == 'given':
        p_values[0] = stag_p[1] - 0.5 * row * u_solution[0]**2 * (A[1] / A[0])**2
    if inlet_p[0] == 'given':
        p_values[0] == inlet_p[1]
    if outlet_p[0] == 'given':
        p_values[p_grid_pts - 1] == outlet_p[1]
    
    p_values[1:p_grid_pts] += alpha * p_solution[1:p_grid_pts]

    #print(u_solution, '\n')
    #print(p_values, '\n')

    residue = abs(u_values - u_solution)
    print('**************** Residue ******************', '\n')
    print(residue)

    if all(residue < 0.00001):
        print('################ Convergence #############', '\n')
        print('!!!!!!!!!! Iterations = ', i, '!!!!!!!!!!!', '\n')
        print('*************** u-velocity solution ****************', '\n')
        print(u_solution, '\n')
        print('*************** pressure solution ****************', '\n')
        print(p_values, '\n')
        break

    else:
        u_values[:] = u_solution

**************** Residue ****************** 

[ 0.54206518  0.69694095  0.97571733  1.62619555]
**************** Residue ****************** 

[ 0.15265236  0.19626733  0.27477426  0.45795709]
**************** Residue ****************** 

[ 0.0879554   0.11308552  0.15831973  0.26386621]
**************** Residue ****************** 

[ 0.05147377  0.06618056  0.09265278  0.1544213 ]
**************** Residue ****************** 

[ 0.02490927  0.0320262   0.04483668  0.07472781]
**************** Residue ****************** 

[ 0.00862254  0.01108612  0.01552057  0.02586762]
**************** Residue ****************** 

[ 0.00030916  0.00039749  0.00055648  0.00092747]
**************** Residue ****************** 

[ 0.00440711  0.00566629  0.00793281  0.01322134]
**************** Residue ****************** 

[ 0.00564914  0.00726318  0.01016845  0.01694741]
**************** Residue ****************** 

[ 0.00538146  0.00691902  0.00968662  0.01614437]
**************** Residue ***************